# labdevices examples

Below are several examples of how to use or test the drivers in the labdevices folder.
Just scroll through and play around.

## General Tests
Use the pyvisa package connecting to all kinds of different devices via GPIB, SERIAL, USB, ETHERNET

In [1]:
import pyvisa

In [2]:
rm = pyvisa.ResourceManager()
resources = rm.list_resources()
print(resources)

('ASRL/dev/ttyS4::INSTR', 'USB0::10893::5984::MY55280218::0::INSTR')


In [ ]:
device = rm.open_resource(
    resources[1],
    timeout=100,
    encoding='ascii',
    parity=pyvisa.constants.Parity.none,
    baud_rate=9600,
    data_bits=8,
    stop_bits=pyvisa.constants.StopBits.one,
    flow_control=pyvisa.constants.VI_ASRL_FLOW_NONE,
    write_termination='\r\n',
    read_termination='\r\n'
)


In [ ]:
device.query('AYT')

In [ ]:
device.query('\x05').split(',')

In [ ]:
device.write('\x03')

In [ ]:
device.close()

## Allied Vision

In [2]:
from labdevices import allied_vision

In [3]:
ccd = allied_vision.Manta(camera_id="DEV_000F314E6DE1")

Available cameras : ['DEV_000F314E6DE1']


In [4]:
ccd.initialize()

Connected to camera : DEV_000F314E6DE1


In [ ]:
for i in ccd.camera.feature_names():
    if "Packet" in i:
        print(i)

In [ ]:
ccd.camera.feature('GVSPAdjustPacketSize').info

In [9]:
ccd.camera.GVSPPacketSize

600

In [10]:
ccd.packetSize

600

## Thorlabs devices
Like e.g. the temperature sensor

In [ ]:
from labdevices import thorlabs

In [ ]:
sensor = thorlabs.TSP01('USB0::4883::33016::M00416750::0::INSTR')

In [ ]:
sensor.initialize()

In [ ]:
sensor.close()

In [ ]:
sensor.temperature_probe1()

## Pfeiffer Vacuum devices
Like vacuum gauge controllers

In [ ]:
from labdevices import pfeiffer_vacuum

In [ ]:
sensor = pfeiffer_vacuum.TPG362()

In [ ]:
sensor.initialize()

In [ ]:
sensor.idn()

In [ ]:
sensor.error_status()

In [ ]:
sensor.pressure_gauge(1)

In [ ]:
sensor.pressure_gauges()

In [ ]:
sensor.close()

In [ ]:
sensor.pressure_unit()

In [ ]:
sensor.temperature()

### Trying to talk to Pfeiffer gauge controllers via Ethernet

In [ ]:
import pyvisa

In [ ]:
rm = pyvisa.ResourceManager()

In [ ]:
rm.list_resources()

In [ ]:
devi = rm.open_resource(
    'TCPIP::192.168.1.10::7776::SOCKET',
    #timeout=100,
    #encoding='ascii',
    #parity=pyvisa.constants.Parity.none,
    #baud_rate=9600,
    #data_bits=8,
    #stop_bits=pyvisa.constants.StopBits.one,
    #flow_control=pyvisa.constants.VI_ASRL_FLOW_NONE,
    write_termination='\r',
    #read_termination='\r\n' 
)

In [ ]:
devi.close()

In [ ]:
devi.query('MV')

## Applied Motion Products
For the control of e.g. the stepper motors of the rotary feedthroughs in the spectrometer chamber.

In [ ]:
from labdevices import applied_motion_products

In [ ]:
# Available IP addresses are 10.0.0.102 and 10.0.0.103. See also the IP table on the wiki.
device = applied_motion_products.STF03D('10.0.0.103')

In [ ]:
device.initialize()

In [ ]:
device.close()

In [ ]:
device.is_moving()

In [ ]:
accel = ''
device.deceleration(accel)
device.acceleration(accel)

In [ ]:
device.speed()

In [ ]:
device.idle_current(0.5)

In [ ]:
device.move_relative(40)

In [ ]:
device.microstep()

## Rohde & Schwarz

In [ ]:
from labdevices import rohde_schwarz
import numpy as np
device = rohde_schwarz.FPC1000('10.0.0.90')


In [ ]:
device.initialize()

In [ ]:
device.close()

In [ ]:
device.query('FORM:DATA?')

In [ ]:
# Acquire the trace and save to file.
# Get trace sometimes times out. I don't know why... Any help is appreciated.
x, y = device.get_trace()
np.savetxt('test.txt', np.column_stack([x,y])) 

In [ ]:
device.query('SYST:ERR:ALL?')

## Keysight

### Oscilloscope

In [3]:
from labdevices import keysight
device = keysight.Oscilloscope(resources[1])

In [4]:
device.initialize()

Connected to:
KEYSIGHT TECHNOLOGIES,DSO-X 3034T,MY55280218,04.06.2015051201


In [5]:
trace = device.get_trace(1)
print(trace)

(array([-9.531250e-07, -9.527250e-07, -9.523250e-07, ...,  1.045675e-06,
        1.046075e-06,  1.046475e-06]), array([-0.74773869, -0.57889447, -0.64321608, ..., -0.0080402 ,
       -0.0080402 , -0.0080402 ]))


In [6]:
device.query(':WAVeform:SOURce?')

'CHAN1'

In [7]:
device.get_preamble(channel=1)

Preamble(data_format=0, data_type=0, points=5000, count=1, x_increment=4e-10, x_origin=-9.53125e-07, x_reference=0, y_increment=0.008040201, y_origin=0.0, y_reference=128)

### Counter

In [8]:
counter = keysight.Counter('10.0.0.120')

In [9]:
counter.initialize()

Connected to:
Agilent Technologies,53230A,MY50000373,01.12-1313.3321-1.19-4.10-127-154-35


In [10]:
counter.get_trigger_mode()

'IMM'

In [11]:
counter.measure_frequency()

100000.054395551

In [12]:
counter.close()